In [1]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-all')).length == 0){
  IPython.toolbar.add_buttons_group([
        {
             'label'   : 'kill and run-all',
             'icon'    : 'fa fa-angle-double-down',
             'callback': function(){
                 IPython.notebook.kernel.restart();
                 $(IPython.events).one('kernel_ready.Kernel',
                                       function(){IPython.notebook.execute_all_cells();});
             }
        }
    ], 'kill-run-all');
}

<IPython.core.display.Javascript object>

In [2]:
# import seaborn as sns
import matplotlib.pyplot as plt
from foof.util import create_psd
%matplotlib inline

In [ ]:
# stim induced
%run ../bin/ei.py ../data/test/ \
    -t 15.0 --stim 0.25 --rate 30 --period 0.5 \
    --I_e 0.0 --I_i 0.8 --I_i_sigma 0.0 --I_e_sigma 0.0 \
    --w_e 0.5 --w_ee 0.1 --w_ii 0.5 \
    --w_ei .3 --w_ie 1.2 --stdp

Starting simulation for duration 0.5 s
95.52 ms (19%) simulated in 10s, estimated 42s remaining.
191.12 ms (38%) simulated in 20s, estimated 32s remaining.
289.54 ms (57%) simulated in 30s, estimated 22s remaining.
0.38144 s (76%) simulated in 40s, estimated 12s remaining.
0.48011 s (96%) simulated in 50s, estimated 2s remaining.
0.5 s (100%) simulated in 52s
Saving results
Starting simulation for duration 0.5 s
48.14 ms (9%) simulated in 10s, estimated 1m 34s remaining.
95.955 ms (19%) simulated in 20s, estimated 1m 24s remaining.
145.4 ms (29%) simulated in 30s, estimated 1m 13s remaining.
191.65 ms (38%) simulated in 40s, estimated 1m 4s remaining.
241.205 ms (48%) simulated in 50s, estimated 54s remaining.
0.5 s (100%) simulated in 51s
Saving results
Starting simulation for duration 0.5 s
31.84333333 ms (6%) simulated in 10s, estimated 2m 27s remaining.
63.68333333 ms (12%) simulated in 20s, estimated 2m 17s remaining.
96.24666667 ms (19%) simulated in 30s, estimated 2m 6s remainin

WARNING  py.warnings: /home/ejp/anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)

  warnings.warn("Mean of empty slice", RuntimeWarning)



Saving results
Starting simulation for duration 0.5 s
23.5675 ms (4%) simulated in 10s, estimated 3m 22s remaining.
47.005 ms (9%) simulated in 20s, estimated 3m 13s remaining.
71.3175 ms (14%) simulated in 30s, estimated 3m 0s remaining.
94.0275 ms (18%) simulated in 40s, estimated 2m 53s remaining.
118.33 ms (23%) simulated in 50s, estimated 2m 41s remaining.
0.5 s (100%) simulated in 52s
Saving results
Starting simulation for duration 0.5 s
18.864 ms (3%) simulated in 10s, estimated 4m 15s remaining.
37.628 ms (7%) simulated in 20s, estimated 4m 6s remaining.
57.034 ms (11%) simulated in 30s, estimated 3m 53s remaining.
75.172 ms (15%) simulated in 40s, estimated 3m 46s remaining.
94.718 ms (18%) simulated in 50s, estimated 3m 34s remaining.


In [6]:
# Unpack
period = 0.5
sel = 1

# result = results[sel]

spikes_e = result['spikes_e']
spikes_stim = result['spikes_stim']
spikes_i = result['spikes_i']
pop_e = result['pop_e']
pop_stim = result['pop_stim']
traces_e = result['traces_e']
traces_i = result['traces_i']
t = 2000

# ------------------------------------------------------------
# Raster full
t_min = period * sel
t_max = t_min + period

figure(figsize=(14, 12))
subplot(311)
plot(spikes_stim.t/ms, spikes_stim.i, 'o', label='', color='k')
xlim(t_min * 1000, t_max * 1000)
ylabel("Neuron #")
legend()

subplot(312)
plot(spikes_e.t/ms, spikes_e.i, 'o', label='E', color='k')
plot(spikes_i.t/ms, spikes_i.i, 'o', label='I', color='r')
xlim(t_min * 1000, t_max * 1000)
ylabel("Neuron #")
legend()


# Raster zoom
figure(figsize=(10, 3))
subplot(211)
plot(spikes_e.t/ms, spikes_e.i, 'o', label='E', color='k')
ylabel("Neuron #")
ylim(0, 200)
xlim(t_max * 1000 - 200, t_max * 1000)
legend()

subplot(212)
plot(spikes_i.t/ms, spikes_i.i, 'o', label='I', color='r')
xlabel("Time (ms)")
ylim(0, 40)
xlim(t_max * 1000 - 200, t_max * 1000)

# ------------------------------------------------------------
# V 
tn = 2000
figure(figsize=(14, 6))
subplot(211)
plot(traces_e.t[tn:]/ms, traces_e.V[:, tn:].T/mvolt, color='k')
# ylim(-90, -50)
ylabel("V")
legend()

subplot(212)
plot(traces_i.t[tn:]/ms, traces_i.V[:, tn:].T/mvolt, color='r')
xlabel("Time (ms)")
legend()

# V hist
figure(figsize=(6,6))
subplot(211)
hist(traces_e.V[0][tn:]/mvolt, label='E (V)', color='k', bins=100, range=(-80, -50))
xlabel("V")
legend()

# ------------------------------------------------------------
# E rate
figure(figsize=(14, 3))
plot(pop_e.t[tn:]/ms, pop_e.rate[tn:]/Hz, color='k', label='E rate')
ylabel('Rate (Hz)')
xlabel("Time (ms)")
legend()

figure(figsize=(14, 3))
plot(pop_stim.t[tn:]/ms, pop_stim.rate[tn:]/Hz, color='k', label='Stim rate')
ylabel('Rate (Hz)')
xlabel("Time (ms)")
legend()

# ------------------------------------------------------------
figure(figsize=(14, 12))
subplot(511)
plot(traces_e.t[tn:]/ms, traces_e.g_s[1:20, tn:].T, label='E (g_s)')
ylabel("E g_s")

subplot(512)
plot(traces_e.t[tn:]/ms, traces_e.g_e[1:20, tn:].T, label='E (g_e)')
ylabel("E g_e")

subplot(513)
plot(traces_e.t[tn:]/ms, traces_e.g_i[1:20, tn:].T, label='E (g_i)')
ylabel("E g_i")

subplot(514)
plot(traces_i.t[tn:]/ms, traces_i.g_i[1:20, tn:].T, label='I (g_i)')
ylabel("I g_i")
# legend(loc='best')

subplot(515)
plot(traces_e.t[tn:]/ms, traces_e.g_ee[1:20, tn:].T, label='E (g_ee)')
ylabel("g_ee")
xlabel("Time (ms)")

# ------------------------------------------------------------
# Spectra
lfp = (np.abs(traces_e.g_s.sum(0)) + np.abs(traces_e.g_ee.sum(0)) + np.abs(traces_e.g_i.sum(0)))
lfp = lfp[1000:]  # Drop initial spike

figure(figsize=(6,6))
subplot(211)
plot(traces_i.t[1000:]/ms, lfp, color='k',  label='LFP')
xlabel("Time (ms)")
ylabel("Sim. LFP (msiemens)")

subplot(212)
fs, spec = create_psd(lfp, 100000)
plot(fs[:60], spec[:60], color='k')
xlabel("Freq (Hz)") 
ylabel("PSD")

NameError: name 'result' is not defined

# 